# Gathering labels

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../common_.py
import sys
sys.path.append("../..")
from common import *

In [3]:
import pyarrow

## Load data

In [4]:
try:
    image_path = snakemake.input.image_path
    moa_path = snakemake.input.moa_path
    output = snakemake.output[0]
except NameError:
    data_root = Path("/data/gent/vo/000/gvo00070/vsc42015/datasets/BBBC021")
    image_path = data_root / "BBBC021_v1_image.csv"
    moa_path = data_root / "BBBC021_v1_moa.csv"
    output = data_root / "labels.parquet"

## meta

In [21]:
image = pandas.read_csv(image_path)
image = image.drop(columns=["Image_PathName_DAPI", "Image_PathName_Tubulin", "Image_PathName_Actin", "Image_FileName_Tubulin", "Image_FileName_Actin"])
image.columns = ["meta_" + col.lower() for col in image.columns]

In [22]:
image.head()

,meta_tablenumber,meta_imagenumber,meta_image_filename_dapi,meta_image_metadata_plate_dapi,meta_image_metadata_well_dapi,meta_replicate,meta_image_metadata_compound,meta_image_metadata_concentration
0,4,233,G10_s1_w1BEDC2073-A983-4B98-95E9-84466707A25D.tif,Week4_27481,G10,1,5-fluorouracil,0.003
1,4,234,G10_s2_w11C3B9BCC-E48F-4C2F-9D31-8F46D8B5B972.tif,Week4_27481,G10,1,5-fluorouracil,0.003
2,4,235,G10_s3_w1F4FCE330-C71C-4CA3-9815-EAF9B9876EB5.tif,Week4_27481,G10,1,5-fluorouracil,0.003
3,4,236,G10_s4_w1747818B4-FFA7-40EE-B0A0-6A5974AF2644.tif,Week4_27481,G10,1,5-fluorouracil,0.003
4,4,473,G10_s1_w10034568D-CC12-43C3-93A9-DC3782099DD3.tif,Week4_27521,G10,2,5-fluorouracil,0.003


In [33]:
moa = pandas.read_csv(moa_path)

In [34]:
labels = image.merge(moa, left_on=("meta_image_metadata_compound", "meta_image_metadata_concentration"), right_on=("compound", "concentration"), how="inner")
labels = labels.drop(columns=["meta_image_metadata_compound", "meta_image_metadata_concentration"])

In [36]:
labels.columns = [c if c.startswith("meta_") else "meta_"+c for c in labels.columns]

In [37]:
pq.write_table(pyarrow.Table.from_pandas(labels), output)

In [44]:
labels[labels["meta_image_metadata_plate_dapi"] == "Week5_28921"]["meta_moa"].value_counts()

Microtubule stabilizers    24
DMSO                       24
Epithelial                 12
Name: meta_moa, dtype: int64

In [49]:
sorted(labels[labels["meta_moa"] == "Protein degradation"]["meta_image_metadata_plate_dapi"].unique())

['Week2_24121',
 'Week2_24141',
 'Week2_24161',
 'Week2_24361',
 'Week2_24381',
 'Week2_24401',
 'Week6_31641',
 'Week6_31661',
 'Week6_31681',
 'Week7_34341',
 'Week7_34381',
 'Week7_34641']